<a href="https://colab.research.google.com/github/rosafilgueira/Seminar_MUIA/blob/main/Colab/lab_exercises/lab_rdd_dataframes_NLTK.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Preparing Enviroment




In [1]:
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q https://dlcdn.apache.org/spark/spark-3.2.1/spark-3.2.1-bin-hadoop3.2.tgz
!tar xf spark-3.2.1-bin-hadoop3.2.tgz
!pip install -q findspark
!pip install pyspark

from pyspark.sql import SparkSession
spark = SparkSession.builder \
    .master("local[*]") \
    .appName("Learning_Spark") \
    .getOrCreate()
sc = spark.sparkContext

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 281.3 MB 46 kB/s 
     |████████████████████████████████| 199 kB 51.2 MB/s 
  Created wheel for pyspark: filename=pyspark-3.3.0-py2.py3-none-any.whl size=281764026 sha256=4fa75ebe7de04fa126eebeb5b7780e7ff1680c4257131d5f8572fd3fd2120f68
  Stored in directory: /root/.cache/pip/wheels/7a/8e/1b/f73a52650d2e5f337708d9f6a1750d451a7349a867f928b885
Successfully built pyspark


# Spark Dataframes (and RDDs) for manipulating text

Type the following commands, for installing different nltk packages that we will use later. 

In [2]:
import nltk
nltk.download('wordnet')
nltk.download('punkt')
nltk.download('stopwords')

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

The aim of this exercise is to learn how to manipulate data using spark dataframes. We are going to work with a subset of the encyclopaedia britannica from the [National Library of Scotland](https://data.nls.uk/data/digitised-collections/encyclopaedia-britannica/). We have downloaded previously the full dataset, ingested the data, and produce a subsample dataset, called *"nls_demo.csv"* (which is the one we are going to use) using a spark-tool called [defoe](https://github.com/alan-turing-institute/defoe/blob/master/docs/nls_demo_examples/nls_demo_individual_queries.md). 

defoe query run to obtain the data: spark-submit --py-files defoe.zip defoe/run_query.py nls_tiny.txt nls defoe.nls.queries.write_pages_df_yml queries/writehdfs.yml -r results -n 34 

Download [nls_demo.csv](https://drive.google.com/file/d/1c2RGeV9enUJWrGjDc4kOESDUybYoQQrC/view?usp=sharing)
And place in this folder

In [9]:
import nltk
import string
from pyspark.sql import SQLContext
sqlContext=SQLContext(spark)

/usr/local/lib/python3.7/dist-packages/pyspark/sql/context.py:114: FutureWarning: Deprecated in 3.0.0. Use SparkSession.builder.getOrCreate() instead.
  FutureWarning,


In [8]:
spark

In [4]:
!ls

nls_demo.csv  spark-3.2.1-bin-hadoop3.2
sample_data   spark-3.2.1-bin-hadoop3.2.tgz


This is the format of the CSV file:
title,edition,year,place,archive_filename,page_filename,page_id,num_pages,type_archive,model,page_string

In [10]:
#Reading the csv file into a dataframe
df= sqlContext.read.csv("./nls_demo.csv", header="true")
df.show(3)

+--------------------+--------------------+----+---------+--------------------+--------------------+-------+----------+------------+-----+------------------+
|               title|             edition|year|    place|    archive_filename|       page_filename|page_id| num_pages|type_archive|model|       page_string|
+--------------------+--------------------+----+---------+--------------------+--------------------+-------+----------+------------+-----+------------------+
|Encyclopaedia Bri...|Second edition, V...|1778|Edinburgh|/Users/rf208/Rese...|alto/188386434.34...|  Page1|       872|        book|  nls|              null|
|Encyclopaedia Bri...|Second edition, V...|1778|Edinburgh|/Users/rf208/Rese...|alto/188386447.34...|  Page2|       872|        book|  nls|              null|
|Encyclopaedia Bri...|Second edition, V...|1778|Edinburgh|/Users/rf208/Rese...|alto/188386460.34...|  Page3|       872|        book|  nls|FiS. 4 ,^-T 'I W ;|
+--------------------+--------------------+----+----

Lets see the first row

In [11]:
df.take(1)

[Row(title='Encyclopaedia Britannica', edition='Second edition, Volume 1, A-AST', year='1778', place='Edinburgh', archive_filename='/Users/rf208/Research/NLS-Fellowship/work/defoe/nls-data-encyclopaediaBritannica/144850370', page_filename='alto/188386434.34.xml', page_id='Page1',  num_pages='872', type_archive='book', model='nls', page_string=None)]

In [12]:
#Filtering by pages that are not null, and grouping by year, and counting the number of pages
df.filter(df.page_string.isNotNull()).select(df.year, df.page_string).groupby(df.year).count().show()

+----+-----+
|year|count|
+----+-----+
|1778| 8918|
|1797|  103|
+----+-----+



In [13]:
# Count the number of pages that belongs to the Second Edition
df.filter(df.page_string.isNotNull()).filter(df.edition.contains('Second')).count()


8918

In [14]:
# Count the number of pages that belongs to the Third Edition
df.filter(df.page_string.isNotNull()).filter(df.edition.contains('Third')).count()

103

In [15]:
#Same that before, but grouping by place
df.filter(df.page_string.isNotNull()).select(df.place, df.page_string).groupby(df.place).count().show()

+---------+-----+
|    place|count|
+---------+-----+
|Edinburgh| 9021|
+---------+-----+



In [16]:
#Grouping by years, but just the ones between 1773 and 1779
df.filter(df.page_string.isNotNull()).filter(df.year.between(1773, 1779)).select(df.place, df.edition, df.page_string, df.year).groupby(df.year).count().show()

+----+-----+
|year|count|
+----+-----+
|1778| 8918|
+----+-----+



In [17]:
df.filter(df.page_string.isNotNull()).filter(df.year.between(1773, 1779)).select(df.place, df.edition, df.page_string, df.year).show()

+---------+--------------------+--------------------+----+
|    place|             edition|         page_string|year|
+---------+--------------------+--------------------+----+
|Edinburgh|Second edition, V...|  FiS. 4 ,^-T 'I W ;|1778|
|Edinburgh|Second edition, V...|                   t|1778|
|Edinburgh|Second edition, V...|ENCYCLOPEDIA BRIT...|1778|
|Edinburgh|Second edition, V...|* ■' M.rg <?■ 1965 #|1778|
|Edinburgh|Second edition, V...|PREFACE. THE mind...|1778|
|Edinburgh|Second edition, V...|E. P R E F A C ha...|1778|
|Edinburgh|Second edition, V...|P R E F A C E. th...|1778|
|Edinburgh|Second edition, V...|PREFACE. ments, w...|1778|
|Edinburgh|Second edition, V...|PREFACE. hoped, w...|1778|
|Edinburgh|Second edition, V...|PREFACE. to affor...|1778|
|Edinburgh|Second edition, V...|"A NEW DICT PON A...|1778|
|Edinburgh|Second edition, V...|"A B [ 2 ] ABA Aa...|1778|
|Edinburgh|Second edition, V...|Abaci, various. *...|1778|
|Edinburgh|Second edition, V...|ABA [ 4 Abano Inf...|177

In [19]:
df.filter(df.page_string.isNotNull()).filter(df.year.between(1776, 1843)).filter(df.edition.startswith("Third")).select(df.place, df.edition, df.page_string, df.year).show()

+---------+--------------------+--------------------+----+
|    place|             edition|         page_string|year|
+---------+--------------------+--------------------+----+
|Edinburgh|Third edition, Vo...|                   I|1797|
|Edinburgh|Third edition, Vo...|ENCYCLOPEDIA BRIT...|1797|
|Edinburgh|Third edition, Vo...|dBntereti in ©tat...|1797|
|Edinburgh|Third edition, Vo...|"T O T H E KING. ...|1797|
|Edinburgh|Third edition, Vo...|"iv DEDICATION. T...|1797|
|Edinburgh|Third edition, Vo...|P R E FAG E. THE ...|1797|
|Edinburgh|Third edition, Vo...|VI PREFACE. BUT t...|1797|
|Edinburgh|Third edition, Vo...|F REF A C E. THES...|1797|
|Edinburgh|Third edition, Vo...|"KNOWLEDGE is eit...|1797|
|Edinburgh|Third edition, Vo...|PREFACE. m make a...|1797|
|Edinburgh|Third edition, Vo...|PREFACE. tends te...|1797|
|Edinburgh|Third edition, Vo...|PREFACE. xi BUT w...|1797|
|Edinburgh|Third edition, Vo...|P R £ £ A C t. if...|1797|
|Edinburgh|Third edition, Vo...|PREFACE; xiii tun...|179

In [20]:
# Now lets create a datafame, which pages are not Null, and selecting just year and page_string columns
newdf=df.filter(df.page_string.isNotNull()).select(df.year, df.page_string)
# And check the Schema of the new dataframe
newdf.printSchema()

root
 |-- year: string (nullable = true)
 |-- page_string: string (nullable = true)



In [21]:
# Count the number of rows
newdf.count()

9021

In [22]:
# Show the first 20 rows
newdf.show()

+----+--------------------+
|year|         page_string|
+----+--------------------+
|1778|  FiS. 4 ,^-T 'I W ;|
|1778|                   t|
|1778|ENCYCLOPEDIA BRIT...|
|1778|* ■' M.rg <?■ 1965 #|
|1778|PREFACE. THE mind...|
|1778|E. P R E F A C ha...|
|1778|P R E F A C E. th...|
|1778|PREFACE. ments, w...|
|1778|PREFACE. hoped, w...|
|1778|PREFACE. to affor...|
|1778|"A NEW DICT PON A...|
|1778|"A B [ 2 ] ABA Aa...|
|1778|Abaci, various. *...|
|1778|ABA [ 4 Abano Inf...|
|1778|"Abarti- culatio ...|
|1778|ABB [ Abbas aged ...|
|1778|"ABB [ 7 ] ABB Ab...|
|1778|"ABB [ 8 ] ABB Ar...|
|1778|ABB [ 9 ] A B D A...|
|1778|"A B D [ io ] A B...|
+----+--------------------+
only showing top 20 rows



In [23]:
# Converting the dataframe to tuples- best suited for processing unstructured data.
pages=newdf.rdd.map(tuple)

In [24]:
pages.take(8)

[('1778', "FiS. 4 ,^-T 'I W ;"),
 ('1778', 't'),
 ('1778',
  'ENCYCLOPEDIA BRITJNNICJ; OR, A DICTIONARY O F ARTS, S C I E N C E S, &c. On a PLAN entirely NEW: BY WHICH, THE DIFFERENT SCIENCES AND ARTS Are digefted into the FORM of Diftinft TREATISES OR SYSTEMS, - COMPREHENDING The HISTORY, THEORY, and PRACTICE, of each, according to the Latefl Difcoveries and Improvements; AND FULL EXPLANATIONS GIVEN OF THE VARIOUS DETACHED PARTS OF KNOWLEDGE, WHETHER Relating to NATURAL and ARTIFICIAL Objedts, or to Matters ECCLESIASTICAL, ► CIVIL, MILITARY, COMMERCIAL, &C. TOGETHER WITH A DESCPJPTION of all the COUNTRIES, CITIES, Principal MOUNTAINS, SEAS, RIVERS, throughout the WORLD; A GENERAL HISTORY, Ancient and Modern> of the different EMPIRES, KINGDOMS, and STATES; AND An ACCOUNT of the LIVES of the mod EMINENT PERSONS in every Nation, from the earlieft ages down to the prefent times. THE WHOLE COMPILED FROM THE WRITINGS OF THE BEST AUTHORS, IN SEVERAL LANGUAGES; THE MOST APPROVED DICTIONARIES,

In [25]:
def sent_TokenizeFunct(x):
    return nltk.sent_tokenize(x)

In [26]:
sentenceTokenizeRDD = pages.map(lambda p: sent_TokenizeFunct(p[1]))

In [28]:
sentenceTokenizeRDD.take(5)

[['FiS.', "4 ,^-T 'I W ;"],
 ['t'],
 ['ENCYCLOPEDIA BRITJNNICJ; OR, A DICTIONARY O F ARTS, S C I E N C E S, &c. On a PLAN entirely NEW: BY WHICH, THE DIFFERENT SCIENCES AND ARTS Are digefted into the FORM of Diftinft TREATISES OR SYSTEMS, - COMPREHENDING The HISTORY, THEORY, and PRACTICE, of each, according to the Latefl Difcoveries and Improvements; AND FULL EXPLANATIONS GIVEN OF THE VARIOUS DETACHED PARTS OF KNOWLEDGE, WHETHER Relating to NATURAL and ARTIFICIAL Objedts, or to Matters ECCLESIASTICAL, ► CIVIL, MILITARY, COMMERCIAL, &C. TOGETHER WITH A DESCPJPTION of all the COUNTRIES, CITIES, Principal MOUNTAINS, SEAS, RIVERS, throughout the WORLD; A GENERAL HISTORY, Ancient and Modern> of the different EMPIRES, KINGDOMS, and STATES; AND An ACCOUNT of the LIVES of the mod EMINENT PERSONS in every Nation, from the earlieft ages down to the prefent times.',
  'THE WHOLE COMPILED FROM THE WRITINGS OF THE BEST AUTHORS, IN SEVERAL LANGUAGES; THE MOST APPROVED DICTIONARIES, AS WELL OF GENERA

In [27]:
def word_TokenizeFunct(x):
    splitted = [word for line in x for word in line.split()]
    return splitted

In [29]:
#wordTokenizeRDD = sentenceTokenizeRDD.map(lambda x: word_TokenizeFunct(x))
wordTokenizeRDD = sentenceTokenizeRDD.map(word_TokenizeFunct)

In [30]:
wordTokenizeRDD.take(10)

[['FiS.', '4', ',^-T', "'I", 'W', ';'],
 ['t'],
 ['ENCYCLOPEDIA',
  'BRITJNNICJ;',
  'OR,',
  'A',
  'DICTIONARY',
  'O',
  'F',
  'ARTS,',
  'S',
  'C',
  'I',
  'E',
  'N',
  'C',
  'E',
  'S,',
  '&c.',
  'On',
  'a',
  'PLAN',
  'entirely',
  'NEW:',
  'BY',
  'WHICH,',
  'THE',
  'DIFFERENT',
  'SCIENCES',
  'AND',
  'ARTS',
  'Are',
  'digefted',
  'into',
  'the',
  'FORM',
  'of',
  'Diftinft',
  'TREATISES',
  'OR',
  'SYSTEMS,',
  '-',
  'COMPREHENDING',
  'The',
  'HISTORY,',
  'THEORY,',
  'and',
  'PRACTICE,',
  'of',
  'each,',
  'according',
  'to',
  'the',
  'Latefl',
  'Difcoveries',
  'and',
  'Improvements;',
  'AND',
  'FULL',
  'EXPLANATIONS',
  'GIVEN',
  'OF',
  'THE',
  'VARIOUS',
  'DETACHED',
  'PARTS',
  'OF',
  'KNOWLEDGE,',
  'WHETHER',
  'Relating',
  'to',
  'NATURAL',
  'and',
  'ARTIFICIAL',
  'Objedts,',
  'or',
  'to',
  'Matters',
  'ECCLESIASTICAL,',
  '►',
  'CIVIL,',
  'MILITARY,',
  'COMMERCIAL,',
  '&C.',
  'TOGETHER',
  'WITH',
  'A',
  'DESCP

In [31]:
def removeStopWordsFunct(x):
    nltk.download('stopwords')
    from nltk.corpus import stopwords
    stop_words=set(stopwords.words('english'))
    filteredSentence = [w for w in x if not w in stop_words]
    return filteredSentence

In [32]:
#stopwordRDD = wordTokenizeRDD.map(lambda x: removeStopWordsFunct(x))
stopwordRDD = wordTokenizeRDD.map(removeStopWordsFunct)

In [33]:
stopwordRDD.take(10)

[['FiS.', '4', ',^-T', "'I", 'W', ';'],
 [],
 ['ENCYCLOPEDIA',
  'BRITJNNICJ;',
  'OR,',
  'A',
  'DICTIONARY',
  'O',
  'F',
  'ARTS,',
  'S',
  'C',
  'I',
  'E',
  'N',
  'C',
  'E',
  'S,',
  '&c.',
  'On',
  'PLAN',
  'entirely',
  'NEW:',
  'BY',
  'WHICH,',
  'THE',
  'DIFFERENT',
  'SCIENCES',
  'AND',
  'ARTS',
  'Are',
  'digefted',
  'FORM',
  'Diftinft',
  'TREATISES',
  'OR',
  'SYSTEMS,',
  '-',
  'COMPREHENDING',
  'The',
  'HISTORY,',
  'THEORY,',
  'PRACTICE,',
  'each,',
  'according',
  'Latefl',
  'Difcoveries',
  'Improvements;',
  'AND',
  'FULL',
  'EXPLANATIONS',
  'GIVEN',
  'OF',
  'THE',
  'VARIOUS',
  'DETACHED',
  'PARTS',
  'OF',
  'KNOWLEDGE,',
  'WHETHER',
  'Relating',
  'NATURAL',
  'ARTIFICIAL',
  'Objedts,',
  'Matters',
  'ECCLESIASTICAL,',
  '►',
  'CIVIL,',
  'MILITARY,',
  'COMMERCIAL,',
  '&C.',
  'TOGETHER',
  'WITH',
  'A',
  'DESCPJPTION',
  'COUNTRIES,',
  'CITIES,',
  'Principal',
  'MOUNTAINS,',
  'SEAS,',
  'RIVERS,',
  'throughout',
  'W

In [34]:
def removePunctuationsFunct(x):
    nltk.download('punkt')
    list_punct=list(string.punctuation)
    filtered = [''.join(c for c in s if c not in list_punct) for s in x] 
    filtered_space = [s for s in filtered if s] #remove empty space 
    return filtered

In [35]:
rmvPunctRDD = stopwordRDD.map(removePunctuationsFunct)

In [36]:
rmvPunctRDD.take(10)

[['FiS', '4', 'T', 'I', 'W', ''],
 [],
 ['ENCYCLOPEDIA',
  'BRITJNNICJ',
  'OR',
  'A',
  'DICTIONARY',
  'O',
  'F',
  'ARTS',
  'S',
  'C',
  'I',
  'E',
  'N',
  'C',
  'E',
  'S',
  'c',
  'On',
  'PLAN',
  'entirely',
  'NEW',
  'BY',
  'WHICH',
  'THE',
  'DIFFERENT',
  'SCIENCES',
  'AND',
  'ARTS',
  'Are',
  'digefted',
  'FORM',
  'Diftinft',
  'TREATISES',
  'OR',
  'SYSTEMS',
  '',
  'COMPREHENDING',
  'The',
  'HISTORY',
  'THEORY',
  'PRACTICE',
  'each',
  'according',
  'Latefl',
  'Difcoveries',
  'Improvements',
  'AND',
  'FULL',
  'EXPLANATIONS',
  'GIVEN',
  'OF',
  'THE',
  'VARIOUS',
  'DETACHED',
  'PARTS',
  'OF',
  'KNOWLEDGE',
  'WHETHER',
  'Relating',
  'NATURAL',
  'ARTIFICIAL',
  'Objedts',
  'Matters',
  'ECCLESIASTICAL',
  '►',
  'CIVIL',
  'MILITARY',
  'COMMERCIAL',
  'C',
  'TOGETHER',
  'WITH',
  'A',
  'DESCPJPTION',
  'COUNTRIES',
  'CITIES',
  'Principal',
  'MOUNTAINS',
  'SEAS',
  'RIVERS',
  'throughout',
  'WORLD',
  'A',
  'GENERAL',
  'HIST

In [37]:
def lemmatizationFunct(x):
    nltk.download('wordnet')
    lemmatizer = nltk.WordNetLemmatizer()
    finalLem = [lemmatizer.lemmatize(s) for s in x]
    return finalLem

In [38]:
lem_wordsRDD = rmvPunctRDD.map(lemmatizationFunct)

In [39]:
lem_wordsRDD.take(10)

Py4JJavaError: ignored